# Assignment 01

## Quesiton 1

In [ ]:
# Importing Libraries
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import csv
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from random import randint

## Part (a)

In [ ]:
# Scraping all the recipies link
base_url = 'https://www.bbc.co.uk'
urls = [[f'{base_url}/food/recipes/a-z/{chr(i)}/{j}#featured-content' for j in range(1, 65)]
        for i in range(ord('a'), ord('z') + 1)]

recipes = []
for url in tqdm(urls):
    for page in url:
        response = requests.get(page)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            page_title = soup.title.string.strip()

            if page_title == "Recipes - BBC Food":
                break

            recipe_links = soup.find_all('a', class_='promo')

            for recipe in recipe_links:
                href = recipe.get('href')
                if href:
                    full_url = base_url + href if href.startswith('/') else href
                    recipes.append(full_url)
            # print(len(recipe_links))
        else:
            break

In [1]:
len(recipes)

9688

In [ ]:
# Save recipes to a CSV file
csv_file = "recipes.csv"
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Recipe URL"])
    for recipe in recipes:
        writer.writerow([recipe])

In [ ]:
# Loading the CSV file
recipes = []
csv_file = "recipes.csv"

with open(csv_file, mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        if row:
            recipes.append(row[0])

### Getting all the recipes

In [ ]:
# Initializing the data
data = []
df = None

In [2]:
# Getting recipie from links
recipe_link = "https://www.bbc.co.uk/food/recipes/areturntotheblackfor_67199"
indices = ["Recipe Name", "Recipe URL", "Ingredients", "Instructions", "Servings", "Prep Time"]

def get_recipe(recipe_link):
    response = requests.get(recipe_link)

    if response.status_code == 200:
        try:
            soup = BeautifulSoup(response.content, 'html.parser')

            recipe_name = soup.find('h1', class_='gel-trafalgar').get_text(strip=True) if soup.find(
                'h1', class_='gel-trafalgar') else None
            ingredients = [ingredient.get_text(strip=True) if ingredient else None
                           for ingredient in soup.find_all('li', class_='recipe-ingredients__list-item')]
            instructions = [step.get_text(strip=True) if step else None for step in soup.find_all(
                'li', class_='recipe-method__list-item')]
            servings = soup.find('p', class_='recipe-metadata__serving').get_text(strip=True) if soup.find(
                'p', class_='recipe-metadata__serving') else None
            prep_time = soup.find('p', class_='recipe-metadata__prep-time').get_text(strip=True) if soup.find(
                'p', class_='recipe-metadata__prep-time') else None
        except Exception as e:
            print(f"\nFailed to get the recipe from {recipe_link}\nError: {e}")
            return None

        return pd.Series([recipe_name, recipe_link, ingredients, instructions, servings, prep_time], index=indices)
    else:
        print(f"\nFailed to get the recipe from {recipe_link}")
        return None


get_recipe(recipe_link)

Recipe Name                          A return to the Black Forest
Recipe URL      https://www.bbc.co.uk/food/recipes/areturntoth...
Ingredients     [225g/8ozdark chocolate(75 per cent cocoa soli...
Instructions    [Pre-heat the oven to 180C/350F/Gas 4., Break ...
Servings                                             Serves 10-12
Prep Time                                               1-2 hours
dtype: object

In [3]:
# Getting all the recipies
i = 0
for recipe in tqdm(recipes):
    if i < 2984:
        i += 1
        continue
    webpage_data = get_recipe(recipe)
    if webpage_data is not None:
        data.append(webpage_data)

df = pd.DataFrame(data, columns=indices)
df

Failed to get the recipe from https://www.bbc.co.uk/food/recipes/gteau_saint_honor_39706
Failed to get the recipe from https://www.bbc.co.uk/food/recipes/italian_style_meatballs_69955
Failed to get the recipe from https://www.bbc.co.uk/food/recipes/pan_fried_red_mullet_48324
Failed to get the recipe from https://www.bbc.co.uk/food/recipes/cullohillrhubarbpie_11635
Failed to get the recipe from https://www.bbc.co.uk/food/recipes/rib-eye_steak_with_smoky_92982
Failed to get the recipe from https://www.bbc.co.uk/food/recipes/seabasswithtomatooli_72150
Failed to get the recipe from https://www.bbc.co.uk/food/recipes/seabass_with_mussels_41984
Failed to get the recipe from https://www.bbc.co.uk/food/recipes/venison_coconut-curried_11281


                                            Recipe Name  ...          Prep Time
0                          A return to the Black Forest  ...          1-2 hours
1                           Aberdeen butteries (Rowies)  ...          1-2 hours
2                                    Ackee and saltfish  ...  less than 30 mins
3     Acqua e sale (Puglian bread salad) with marina...  ...  less than 30 mins
4            Acqua pazza with hake, fennel and tomatoes  ...  less than 30 mins
...                                                 ...  ...                ...
9675                            Za’atar cod with relish  ...  less than 30 mins
9676                                            Zeppole  ...          1-2 hours
9677                          Zeppole festive doughnuts  ...  less than 30 mins
9678                          Zesty chilli tiger prawns  ...  less than 30 mins
9679                                       Zoe’s jollof  ...  less than 30 mins

[9680 rows x 6 columns]

In [4]:
# Removing the rows duplicates
df.drop_duplicates(inplace=True)

# add a column to the dataframe to store the recipe id
df['Recipe ID'] = range(1, len(df) + 1)

# Move the 'Recipe ID' column to the first position
recipe_id = df.pop('Recipe ID')
df.insert(0, 'Recipe ID', recipe_id)

      Recipe ID  ...          Prep Time
0             1  ...          1-2 hours
1             2  ...          1-2 hours
2             3  ...  less than 30 mins
3             4  ...  less than 30 mins
4             5  ...  less than 30 mins
...         ...  ...                ...
9674       9675  ...  less than 30 mins
9675       9676  ...          1-2 hours
9676       9677  ...  less than 30 mins
9677       9678  ...  less than 30 mins
9678       9679  ...  less than 30 mins

[9679 rows x 7 columns]

In [ ]:
# Saving the df to a CSV file
df.to_csv("recipes_data.csv", index=False)

In [5]:
# Loading the CSV file
df = pd.read_csv("recipes_data.csv")
df

      Recipe ID  ...          Prep Time
0             1  ...          1-2 hours
1             2  ...          1-2 hours
2             3  ...  less than 30 mins
3             4  ...  less than 30 mins
4             5  ...  less than 30 mins
...         ...  ...                ...
9674       9675  ...  less than 30 mins
9675       9676  ...          1-2 hours
9676       9677  ...  less than 30 mins
9677       9678  ...  less than 30 mins
9678       9679  ...  less than 30 mins

[9679 rows x 7 columns]

## Part (b)

In [ ]:
# Initialising the variables
ingredients_df = pd.DataFrame(columns=["Recipe ID", "Ingredient Phrase", "Ingredients"])

In [6]:
# NER on the ingredients
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("Dizex/InstaFoodRoBERTa-NER")
model = AutoModelForTokenClassification.from_pretrained("Dizex/InstaFoodRoBERTa-NER").to(device)

ner = pipeline("ner", model=model, tokenizer=tokenizer, device='mps' if torch.backends.mps.is_available() else 'cpu')

def ner_result(text):
    # Get the ingredients from doc
    ingredients = ner(text)
    return ingredients

def convert_entities_to_list(text, entities: list[dict]) -> list[str]:
    ents = []
    for ent in entities:
        e = {"start": ent["start"], "end": ent["end"], "label": ent["entity"]}
        if ents and -1 <= ent["start"] - ents[-1]["end"] <= 1 and ents[-1]["label"] == e["label"]:
            ents[-1]["end"] = e["end"]
            continue
        ents.append(e)

    return [text[e["start"]:e["end"]] for e in ents]

def get_ingredients(text):
    ingredients = []
    for item in eval(text):
        ingredient = " ".join(convert_entities_to_list(item, ner_result(item)))
        if ingredient.strip() != "":
            ingredients.append(ingredient)

    return ingredients


print(df['Ingredients'][1])
# print(convert_entities_to_list(df["Ingredients"][0], ner_result(df['Ingredients'][0])))
#
# print("\n\n")
print(get_ingredients(df['Ingredients'][1]))

['500g/1lb 2oz strong plain flour, plus extra for dusting', '1 x 7g sachet easy-blend driedyeast', '1 tbsp soft lightbrown sugar', '1 tbsp seasaltflakes', '350ml/12¼fl oz warm water', 'vegetable oil, for greasing', '275g/9¾ozbutter, plus extra to serve', '100g/3½ozlard', 'ready-madejam, to serve']
['plain flour', 'sachet', 'lightbrown sugar', 'seasaltflakes', 'water', 'vegetable oil', 'butter', 'ozlard']


In [7]:
# Doing NER on the whole dataframe
total_operations = len(df)
for index, row in tqdm(df.iterrows(), total=total_operations):
    ingredients = get_ingredients(row['Ingredients'])
    new_row = pd.Series([row["Recipe ID"], row['Ingredients'], ingredients], index=ingredients_df.columns)
    ingredients_df = pd.concat([ingredients_df, new_row.to_frame().T], ignore_index=True)
    # ingredients_df = ingredients_df.append({"Recipe ID": row["Recipe ID"], "Ingredient Phrase": row['Ingredients'],
    #                                         "Ingredients": ingredients}, ignore_index=True)
    # break
ingredients_df

     Recipe ID  ...                                        Ingredients
0            1  ...  [dark chocolate cocoa, s, morellocherries, che...
1            2  ...  [plain flour, sachet, lightbrown sugar, seasal...
2            3  ...  [able oil, 2onions, garlic, pinchthymeleaves, ...
3            4  ...  [onions, softbrown sugar, red wine vinegar bal...
4            5  ...  [2hakefillets, 1fennelbulb, garliccloves, redc...
...        ...  ...                                                ...
9674      9675  ...  [Za’atar, tbspolive oil, primecodfillet loin f...
9675      9676  ...  [all-purposepotatoes, driedyeast, water, plain...
9676      9677  ...  [plain flour, tbspolive oil, runnyhoney, limon...
9677      9678  ...  [groundnut oil orvegetable oil, 5garliccloves,...
9678      9679  ...  [thighs drumsticks, largeonion, Scotch bonnet ...

[9679 rows x 3 columns]

In [ ]:
# Saving the df to a CSV file
ingredients_df.to_csv("ingredients_data.csv", index=False)

In [ ]:
# Loading the CSV file
ingredients_df = pd.read_csv("ingredients_data.csv")

## Part (c)

In [8]:
# Modifying the df to required format
recipe_ingredient_df = pd.DataFrame(columns=["Recipe ID", "Ingredient"])

random_sample = df.sample(n=100, random_state=1)

new_rows = []
for index, row in tqdm(random_sample.iterrows(), total=random_sample.shape[0]):
    recipe_id = row["Recipe ID"]
    ingredients = ingredients_df[ingredients_df["Recipe ID"] == recipe_id]["Ingredients"].values[0]
    for ingredient in ingredients:
        new_rows.append({"Recipe ID": recipe_id, "Ingredient": ingredient})

recipe_ingredient_df = pd.DataFrame(new_rows)

recipe_ingredient_df

      Recipe ID     Ingredient
0          3566  vegetable oil
1          3566  goats’ cheese
2          3566          flour
3          3566    breadcrumbs
4          3566      freshpeas
...         ...            ...
1295       5229            egg
1296       5229      mincemeat
1297       5229     ozmarzipan
1298       5229    icing sugar
1299       5229   brandy cream

[1300 rows x 2 columns]

In [ ]:
# Saving the df to a CSV file
recipe_ingredient_df.to_csv("recipe_ingredient_data.csv", index=False)

In [ ]:
# Loading the CSV file
recipe_ingredient_df = pd.read_csv("recipe_ingredient_data.csv")